# Pipeline



In [49]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [50]:
train = pd.read_csv("Training_Data_0611N.csv")
test = pd.read_csv("Evaluation_Data_0611_Nv.csv")

In [51]:
test.head()

,cust_id,count_accts,cm_age,flag_top_ed_spender,flag_cust_fee_paid_6m,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,pre6m_cust_disc_amt,pre6m_cust_disc_cnt,pre6m_cust_outbound_amt,pre6m_cust_online_amt,pre6m_cust_online_cnt,pre6m_cust_travel_amt,pre6m_cust_travel_cnt,pre6m_cust_retail_amt,pre6m_cust_retail_cnt,pre6m_cust_myca_active,pre6m_cust_mob_logins,pre6m_total_mc_trs,acq_sub_chn,acq_type_grp,fee_type_grp,Cust_tenure,pre6m_spend_active_ind,highly_utilized_ind,min_pay_ind,paid_in_full_ind,sum_total_line_amt,direct_debit_ind,referrals,spillover,self_accts,Customer Low Quality indicator,cdss_most_rcnt_prob,cust_max_credit_12m_amt,cust_max_remit_12m_amt,cbr_3_score,cnsumr_chrg_actv_cust_cnt,cnsumr_chrg_avg_credit_12m_amt,cnsumr_lend_actv_cust_cnt,cnsumr_lend_tot_util_ratio,sow_revol_avg_paydown_pct,sow_tot_annual_ext_pmt_amt,sow_tot_revol_bal_amt,sow_tot_revol_cnt,sow_tot_trans_bal_amt,sow_tot_trans_cnt
0,569986,1,37,0.0,0,"9,868",760,777,174,"9,091",586,"1,705","2,286",280,"3,219",268,"2,668",240,5,9,474,Paid Search,external,no_fee,10.0,1.0,1.0,0.0,0.0,"46,000",0.0,0.0,0.0,0.0,0.0,0.048,"1,880","1,875",833.0,0.0,-,1.0,1.055513,0.0,-,-,0.0,-,0.0
1,569987,2,33,0.0,0,-,0,-,0,-,0,-,-,0,-,0,-,0,0,0,(0),MGM,external,reduced_fee,1.0,0.0,0.0,0.0,0.0,"5,133",0.0,0.0,0.0,0.0,0.0,NaN,-,-,NaN,NaN,-,NaN,NaN,NaN,-,-,NaN,-,NaN
2,569988,2,40,1.0,0,"2,401",302,"1,045",166,"1,357",136,289,483,96,220,18,986,176,5,5,2,MGM,spillover,no_fee,10.0,1.0,0.0,0.0,1.0,"69,600",0.0,0.0,0.0,0.0,0.0,0.000,739,618,1251.0,0.0,43,2.0,0.029297,0.0,-,-,0.0,-,0.0
3,569989,1,34,1.0,1,"9,836",856,829,190,"9,007",666,"2,104","2,545",292,"2,109",254,"1,921",274,6,20,170,MGM,external,reduced_fee,14.0,1.0,0.0,0.0,0.0,-,1.0,0.0,0.0,0.0,0.0,0.000,"1,864","1,851",1004.0,1.0,829,0.0,0.000000,0.0,-,-,0.0,-,0.0
4,569990,1,28,0.0,0,"2,351",260,584,90,"1,767",170,63,282,56,127,26,"1,878",124,3,3,(9),MGM,external,no_fee,7.0,1.0,0.0,0.0,1.0,"9,600",0.0,0.0,0.0,0.0,0.0,0.000,752,731,1293.0,0.0,-,1.0,0.200658,0.0,-,-,0.0,-,0.0


In [52]:
test.isnull().sum()

cust_id                             0
count_accts                         0
cm_age                              0
flag_top_ed_spender                15
flag_cust_fee_paid_6m               0
pre6m_cust_spend                    0
pre6m_cust_roc_cnt                  0
pre6m_cust_non_disc_amt             0
pre6m_cust_non_disc_cnt             0
pre6m_cust_disc_amt                 0
pre6m_cust_disc_cnt                 0
pre6m_cust_outbound_amt             0
pre6m_cust_online_amt               0
pre6m_cust_online_cnt               0
pre6m_cust_travel_amt               0
pre6m_cust_travel_cnt               0
pre6m_cust_retail_amt               0
pre6m_cust_retail_cnt               0
pre6m_cust_myca_active              0
pre6m_cust_mob_logins               0
pre6m_total_mc_trs                  0
acq_sub_chn                       151
acq_type_grp                      142
fee_type_grp                        0
Cust_tenure                        78
pre6m_spend_active_ind             78
highly_utili

In [53]:
train.dtypes

cust_id                           float64
profitable_flag                   float64
count_accts                       float64
cm_age                            float64
flag_top_ed_spender               float64
flag_cust_fee_paid_6m             float64
pre6m_cust_spend                   object
pre6m_cust_roc_cnt                float64
pre6m_cust_non_disc_amt            object
pre6m_cust_non_disc_cnt           float64
pre6m_cust_disc_amt                object
pre6m_cust_disc_cnt               float64
pre6m_cust_outbound_amt            object
pre6m_cust_online_amt              object
pre6m_cust_online_cnt             float64
pre6m_cust_travel_amt              object
pre6m_cust_travel_cnt             float64
pre6m_cust_retail_amt              object
pre6m_cust_retail_cnt             float64
pre6m_cust_myca_active            float64
pre6m_cust_mob_logins             float64
pre6m_total_mc_trs                 object
acq_sub_chn                        object
acq_type_grp                      

In [54]:
train.shape

(19599, 50)

In [55]:
train.isnull().sum()

cust_id                             4
profitable_flag                     4
count_accts                         4
cm_age                              4
flag_top_ed_spender                52
flag_cust_fee_paid_6m               4
pre6m_cust_spend                    4
pre6m_cust_roc_cnt                  4
pre6m_cust_non_disc_amt             4
pre6m_cust_non_disc_cnt             4
pre6m_cust_disc_amt                 4
pre6m_cust_disc_cnt                 4
pre6m_cust_outbound_amt             4
pre6m_cust_online_amt               4
pre6m_cust_online_cnt               4
pre6m_cust_travel_amt               4
pre6m_cust_travel_cnt               4
pre6m_cust_retail_amt               4
pre6m_cust_retail_cnt               4
pre6m_cust_myca_active              4
pre6m_cust_mob_logins               4
pre6m_total_mc_trs                  4
acq_sub_chn                       906
acq_type_grp                      876
fee_type_grp                        4
Cust_tenure                       240
pre6m_spend_

### Imbalanced Dataset

In [56]:
train['profitable_flag'].value_counts(normalize=True)

0.0    0.814034
1.0    0.185966
Name: profitable_flag, dtype: float64

### Object and categorical columns

In [57]:
# storing the list of columns given as objects which needs to be converted
object_columns = list(train.select_dtypes(['object']).columns)

In [58]:
object_columns

['pre6m_cust_spend',
 'pre6m_cust_non_disc_amt',
 'pre6m_cust_disc_amt',
 'pre6m_cust_outbound_amt',
 'pre6m_cust_online_amt',
 'pre6m_cust_travel_amt',
 'pre6m_cust_retail_amt',
 'pre6m_total_mc_trs',
 'acq_sub_chn',
 'acq_type_grp',
 'fee_type_grp',
 'sum_total_line_amt',
 'cust_max_credit_12m_amt',
 'cust_max_remit_12m_amt',
 'cnsumr_chrg_avg_credit_12m_amt',
 'sow_tot_annual_ext_pmt_amt',
 'sow_tot_revol_bal_amt',
 'sow_tot_trans_bal_amt']

In [59]:
categorical_cols = ['acq_sub_chn','acq_type_grp','fee_type_grp']

In [60]:
# HAVING A LOOK AT COLUMNS WHICH ARE having 2 values atleast

train.columns[train.nunique() <=2]

Index(['profitable_flag', 'flag_top_ed_spender', 'flag_cust_fee_paid_6m',
       'pre6m_spend_active_ind', 'highly_utilized_ind', 'min_pay_ind',
       'paid_in_full_ind', 'direct_debit_ind',
       'Customer Low Quality indicator'],
      dtype='object')

In [61]:
train.head()

,cust_id,profitable_flag,count_accts,cm_age,flag_top_ed_spender,flag_cust_fee_paid_6m,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,pre6m_cust_disc_amt,pre6m_cust_disc_cnt,pre6m_cust_outbound_amt,pre6m_cust_online_amt,pre6m_cust_online_cnt,pre6m_cust_travel_amt,pre6m_cust_travel_cnt,pre6m_cust_retail_amt,pre6m_cust_retail_cnt,pre6m_cust_myca_active,pre6m_cust_mob_logins,pre6m_total_mc_trs,acq_sub_chn,acq_type_grp,fee_type_grp,Cust_tenure,pre6m_spend_active_ind,highly_utilized_ind,min_pay_ind,paid_in_full_ind,sum_total_line_amt,direct_debit_ind,referrals,spillover,self_accts,Customer Low Quality indicator,cdss_most_rcnt_prob,cust_max_credit_12m_amt,cust_max_remit_12m_amt,cbr_3_score,cnsumr_chrg_actv_cust_cnt,cnsumr_chrg_avg_credit_12m_amt,cnsumr_lend_actv_cust_cnt,cnsumr_lend_tot_util_ratio,sow_revol_avg_paydown_pct,sow_tot_annual_ext_pmt_amt,sow_tot_revol_bal_amt,sow_tot_revol_cnt,sow_tot_trans_bal_amt,sow_tot_trans_cnt
0,550391.0,0.0,1.0,74.0,1.0,0.0,"1,518",122.0,811,96.0,707,26.0,-,606,36.0,28,2.0,814,98.0,9.0,203.0,(2),Unreferred,external,no_fee,63.0,1.0,0.0,0.0,1.0,"80,000",1.0,1.0,0.0,0.0,0.0,0.000,482,418,1376.0,0.0,-,1.0,0.034044,0.0000,991,-,0.0,-,3.0
1,550392.0,0.0,1.0,64.0,0.0,0.0,"4,807",354.0,"1,052",82.0,"3,756",272.0,673,"3,164",456.0,"2,291",44.0,"1,265",116.0,0.0,0.0,15,In House,others,no_fee,203.0,1.0,0.0,0.0,1.0,"140,000",1.0,1.0,1.0,0.0,0.0,0.000,"2,165","2,165",1300.0,0.0,7,1.0,0.044086,0.0877,2,24,2.0,-,0.0
2,550393.0,0.0,1.0,75.0,1.0,0.0,"2,361",204.0,"1,351",118.0,"1,010",86.0,237,693,152.0,-,0.0,"1,623",166.0,9.0,18.0,(20),NaN,NaN,full_fee,201.0,1.0,0.0,0.0,1.0,"32,800",0.0,0.0,0.0,0.0,0.0,0.000,703,687,1376.0,0.0,-,1.0,0.039613,0.0000,"4,828",-,0.0,387,3.0
3,550394.0,0.0,2.0,73.0,0.0,1.0,"5,452",142.0,"1,195",50.0,"4,257",92.0,803,"1,618",64.0,"2,760",54.0,"1,605",54.0,9.0,23.0,30,NaN,NaN,full_fee,286.0,1.0,0.0,0.0,1.0,"100,000",1.0,0.0,0.0,0.0,0.0,0.000,"1,225","1,225",1372.0,1.0,493,1.0,0.001200,0.0000,"4,598",-,0.0,295,4.0
4,550395.0,0.0,2.0,68.0,0.0,0.0,170,34.0,88,20.0,82,14.0,10,10,4.0,-,0.0,66,18.0,0.0,0.0,(9),Affiliates,spillover,no_fee,123.0,1.0,0.0,0.0,1.0,"20,800",1.0,0.0,0.0,0.0,0.0,0.001,51,35,1338.0,0.0,-,2.0,0.003378,0.0168,"2,176","2,606",5.0,143,13.0


In [62]:
# Removed square brackets from pre6m_total_mc_trs

train['pre6m_total_mc_trs'] = train['pre6m_total_mc_trs'].str.extract('(\d+)', expand=False)

In [63]:
comma_cols = ['pre6m_cust_spend','pre6m_cust_non_disc_amt','pre6m_cust_disc_amt','pre6m_cust_online_amt','pre6m_cust_travel_amt','pre6m_cust_retail_amt','sum_total_line_amt','cust_max_credit_12m_amt','cust_max_remit_12m_amt','sow_tot_annual_ext_pmt_amt']

In [64]:
convert_cols = [f for f in object_columns if f not in categorical_cols]

In [65]:
for f in convert_cols:
    train[str(f)] = train[str(f)].apply(lambda x: str(x).replace(',', ''))

In [66]:
# for f in convert_cols:
#     train[str(f)] = train[str(f)].astype(int)

In [67]:
for cols in convert_cols:
    
    train[str(cols)] = pd.to_numeric(train[str(cols)],errors = 'coerce')

In [68]:
train.head()

,cust_id,profitable_flag,count_accts,cm_age,flag_top_ed_spender,flag_cust_fee_paid_6m,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,pre6m_cust_disc_amt,pre6m_cust_disc_cnt,pre6m_cust_outbound_amt,pre6m_cust_online_amt,pre6m_cust_online_cnt,pre6m_cust_travel_amt,pre6m_cust_travel_cnt,pre6m_cust_retail_amt,pre6m_cust_retail_cnt,pre6m_cust_myca_active,pre6m_cust_mob_logins,pre6m_total_mc_trs,acq_sub_chn,acq_type_grp,fee_type_grp,Cust_tenure,pre6m_spend_active_ind,highly_utilized_ind,min_pay_ind,paid_in_full_ind,sum_total_line_amt,direct_debit_ind,referrals,spillover,self_accts,Customer Low Quality indicator,cdss_most_rcnt_prob,cust_max_credit_12m_amt,cust_max_remit_12m_amt,cbr_3_score,cnsumr_chrg_actv_cust_cnt,cnsumr_chrg_avg_credit_12m_amt,cnsumr_lend_actv_cust_cnt,cnsumr_lend_tot_util_ratio,sow_revol_avg_paydown_pct,sow_tot_annual_ext_pmt_amt,sow_tot_revol_bal_amt,sow_tot_revol_cnt,sow_tot_trans_bal_amt,sow_tot_trans_cnt
0,550391.0,0.0,1.0,74.0,1.0,0.0,1518.0,122.0,811.0,96.0,707.0,26.0,NaN,606.0,36.0,28.0,2.0,814.0,98.0,9.0,203.0,2.0,Unreferred,external,no_fee,63.0,1.0,0.0,0.0,1.0,80000.0,1.0,1.0,0.0,0.0,0.0,0.000,482.0,418.0,1376.0,0.0,NaN,1.0,0.034044,0.0000,991.0,NaN,0.0,NaN,3.0
1,550392.0,0.0,1.0,64.0,0.0,0.0,4807.0,354.0,1052.0,82.0,3756.0,272.0,673.0,3164.0,456.0,2291.0,44.0,1265.0,116.0,0.0,0.0,15.0,In House,others,no_fee,203.0,1.0,0.0,0.0,1.0,140000.0,1.0,1.0,1.0,0.0,0.0,0.000,2165.0,2165.0,1300.0,0.0,7.0,1.0,0.044086,0.0877,2.0,24.0,2.0,NaN,0.0
2,550393.0,0.0,1.0,75.0,1.0,0.0,2361.0,204.0,1351.0,118.0,1010.0,86.0,237.0,693.0,152.0,NaN,0.0,1623.0,166.0,9.0,18.0,20.0,NaN,NaN,full_fee,201.0,1.0,0.0,0.0,1.0,32800.0,0.0,0.0,0.0,0.0,0.0,0.000,703.0,687.0,1376.0,0.0,NaN,1.0,0.039613,0.0000,4828.0,NaN,0.0,387.0,3.0
3,550394.0,0.0,2.0,73.0,0.0,1.0,5452.0,142.0,1195.0,50.0,4257.0,92.0,803.0,1618.0,64.0,2760.0,54.0,1605.0,54.0,9.0,23.0,30.0,NaN,NaN,full_fee,286.0,1.0,0.0,0.0,1.0,100000.0,1.0,0.0,0.0,0.0,0.0,0.000,1225.0,1225.0,1372.0,1.0,493.0,1.0,0.001200,0.0000,4598.0,NaN,0.0,295.0,4.0
4,550395.0,0.0,2.0,68.0,0.0,0.0,170.0,34.0,88.0,20.0,82.0,14.0,10.0,10.0,4.0,NaN,0.0,66.0,18.0,0.0,0.0,9.0,Affiliates,spillover,no_fee,123.0,1.0,0.0,0.0,1.0,20800.0,1.0,0.0,0.0,0.0,0.0,0.001,51.0,35.0,1338.0,0.0,NaN,2.0,0.003378,0.0168,2176.0,2606.0,5.0,143.0,13.0


In [69]:
train.dtypes

cust_id                           float64
profitable_flag                   float64
count_accts                       float64
cm_age                            float64
flag_top_ed_spender               float64
flag_cust_fee_paid_6m             float64
pre6m_cust_spend                  float64
pre6m_cust_roc_cnt                float64
pre6m_cust_non_disc_amt           float64
pre6m_cust_non_disc_cnt           float64
pre6m_cust_disc_amt               float64
pre6m_cust_disc_cnt               float64
pre6m_cust_outbound_amt           float64
pre6m_cust_online_amt             float64
pre6m_cust_online_cnt             float64
pre6m_cust_travel_amt             float64
pre6m_cust_travel_cnt             float64
pre6m_cust_retail_amt             float64
pre6m_cust_retail_cnt             float64
pre6m_cust_myca_active            float64
pre6m_cust_mob_logins             float64
pre6m_total_mc_trs                float64
acq_sub_chn                        object
acq_type_grp                      

In [70]:
duplicate = train[train.duplicated()] 

In [71]:
train.drop_duplicates(keep=False,inplace=True) 

In [72]:
# train.to_csv('train_float_nan.csv',index=False)

In [73]:
f = lambda x: int(x);
train['profitable_flag'] = train['profitable_flag'].apply(f)

In [74]:
# FILLING THE CATEGORICAL COLUMNS WITH NAN CATEGORY

for i,col in enumerate(categorical_cols):
    train[str(col)].fillna("NaN"+str(i),inplace=True)

In [75]:
train[categorical_cols].isnull().sum()

acq_sub_chn     0
acq_type_grp    0
fee_type_grp    0
dtype: int64

In [76]:
train['acq_sub_chn'].unique()

array(['Unreferred', 'In House', 'NaN0', 'Affiliates', 'MGM', 'Email',
       'Partner', 'Non-PDB (Personalized)', 'Outsourced',
       'PDB (Personalized)', 'Interactive', 'Media', 'Search', 'Direct',
       'Partnerships - Other', 'Partnerships - In Store', 'Paid Search',
       'Personalization', 'E-mails', 'E-Mails', 'Natural Referral',
       'Proprietary - Direct Sales', 'DM', 'Outbound', 'Unaddressed',
       'Natural Search'], dtype=object)

In [77]:
train['acq_type_grp'].unique()

array(['external', 'others', 'NaN1', 'spillover'], dtype=object)

In [78]:
train['acq_type_grp'].replace('spillover','spill',inplace=True)

In [79]:
train['fee_type_grp'].unique()

array(['no_fee', 'full_fee', 'reduced_fee', 'unknown'], dtype=object)

### PREDICTORS AND LABELS

In [80]:
y = train['profitable_flag']

In [81]:
x = train.iloc[:,2:]

numerical_cols = [f for f in list(x.columns) if f not in categorical_cols]

In [82]:
x.head()

,count_accts,cm_age,flag_top_ed_spender,flag_cust_fee_paid_6m,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,pre6m_cust_disc_amt,pre6m_cust_disc_cnt,pre6m_cust_outbound_amt,pre6m_cust_online_amt,pre6m_cust_online_cnt,pre6m_cust_travel_amt,pre6m_cust_travel_cnt,pre6m_cust_retail_amt,pre6m_cust_retail_cnt,pre6m_cust_myca_active,pre6m_cust_mob_logins,pre6m_total_mc_trs,acq_sub_chn,acq_type_grp,fee_type_grp,Cust_tenure,pre6m_spend_active_ind,highly_utilized_ind,min_pay_ind,paid_in_full_ind,sum_total_line_amt,direct_debit_ind,referrals,spillover,self_accts,Customer Low Quality indicator,cdss_most_rcnt_prob,cust_max_credit_12m_amt,cust_max_remit_12m_amt,cbr_3_score,cnsumr_chrg_actv_cust_cnt,cnsumr_chrg_avg_credit_12m_amt,cnsumr_lend_actv_cust_cnt,cnsumr_lend_tot_util_ratio,sow_revol_avg_paydown_pct,sow_tot_annual_ext_pmt_amt,sow_tot_revol_bal_amt,sow_tot_revol_cnt,sow_tot_trans_bal_amt,sow_tot_trans_cnt
0,1.0,74.0,1.0,0.0,1518.0,122.0,811.0,96.0,707.0,26.0,NaN,606.0,36.0,28.0,2.0,814.0,98.0,9.0,203.0,2.0,Unreferred,external,no_fee,63.0,1.0,0.0,0.0,1.0,80000.0,1.0,1.0,0.0,0.0,0.0,0.000,482.0,418.0,1376.0,0.0,NaN,1.0,0.034044,0.0000,991.0,NaN,0.0,NaN,3.0
1,1.0,64.0,0.0,0.0,4807.0,354.0,1052.0,82.0,3756.0,272.0,673.0,3164.0,456.0,2291.0,44.0,1265.0,116.0,0.0,0.0,15.0,In House,others,no_fee,203.0,1.0,0.0,0.0,1.0,140000.0,1.0,1.0,1.0,0.0,0.0,0.000,2165.0,2165.0,1300.0,0.0,7.0,1.0,0.044086,0.0877,2.0,24.0,2.0,NaN,0.0
2,1.0,75.0,1.0,0.0,2361.0,204.0,1351.0,118.0,1010.0,86.0,237.0,693.0,152.0,NaN,0.0,1623.0,166.0,9.0,18.0,20.0,NaN0,NaN1,full_fee,201.0,1.0,0.0,0.0,1.0,32800.0,0.0,0.0,0.0,0.0,0.0,0.000,703.0,687.0,1376.0,0.0,NaN,1.0,0.039613,0.0000,4828.0,NaN,0.0,387.0,3.0
3,2.0,73.0,0.0,1.0,5452.0,142.0,1195.0,50.0,4257.0,92.0,803.0,1618.0,64.0,2760.0,54.0,1605.0,54.0,9.0,23.0,30.0,NaN0,NaN1,full_fee,286.0,1.0,0.0,0.0,1.0,100000.0,1.0,0.0,0.0,0.0,0.0,0.000,1225.0,1225.0,1372.0,1.0,493.0,1.0,0.001200,0.0000,4598.0,NaN,0.0,295.0,4.0
4,2.0,68.0,0.0,0.0,170.0,34.0,88.0,20.0,82.0,14.0,10.0,10.0,4.0,NaN,0.0,66.0,18.0,0.0,0.0,9.0,Affiliates,spill,no_fee,123.0,1.0,0.0,0.0,1.0,20800.0,1.0,0.0,0.0,0.0,0.0,0.001,51.0,35.0,1338.0,0.0,NaN,2.0,0.003378,0.0168,2176.0,2606.0,5.0,143.0,13.0


In [83]:
# FILLING THE MISSING NUMERICAL VALUES WITH -1

for col in numerical_cols:
    x[str(col)].fillna(-1,inplace=True)

### CORRELATION ANALYSIS OF GROUPS

In [355]:
grp1 = [f for f in list(x.columns) if f[0:10]=='pre6m_cust']
grp1.append('pre6m_total_mc_trs')
grp2 = ['Cust_tenure','pre6m_spend_active_ind','highly_utilized_ind','min_pay_ind','paid_in_full_ind']
grp3 = ['direct_debit_ind','referrals','spillover','self_accts','Customer Low Quality indicator']
grp4 = ['cdss_most_rcnt_prob','cbr_3_score','cnsumr_chrg_actv_cust_cnt','cnsumr_lend_actv_cust_cnt','cnsumr_lend_tot_util_ratio','sow_revol_avg_paydown_pct','sow_tot_revol_cnt','sow_tot_trans_cnt']

df1 = x[grp1]
df2 = x[grp2]
df3 = x[grp3]
df4 = x[grp4]

In [196]:
df1.corr()

# pre6m_cust_spend - pre6m_cust_online_amt, pre6m_cust_retail_amt,
# pre6m_cust_roc_cnt - pre6m_cust_disc_cnt, pre6m_cust_online_cnt, pre6m_cust_retail_cnt
# pre6m_cust_disc_amt - pre6m_cust_spend, pre6m_cust_online_amt, pre6m_cust_retail_amt
# 

,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,pre6m_cust_disc_amt,pre6m_cust_disc_cnt,pre6m_cust_outbound_amt,pre6m_cust_online_amt,pre6m_cust_online_cnt,pre6m_cust_travel_amt,pre6m_cust_travel_cnt,pre6m_cust_retail_amt,pre6m_cust_retail_cnt,pre6m_cust_myca_active,pre6m_cust_mob_logins,pre6m_total_mc_trs
pre6m_cust_spend,1.000000,0.492282,0.523949,0.298131,0.973711,0.475038,0.655679,0.824611,0.421737,0.601484,0.308282,0.808007,0.403148,0.260918,0.126043,0.358096
pre6m_cust_roc_cnt,0.492282,1.000000,0.290233,0.584062,0.470719,0.971288,0.357307,0.500508,0.822616,0.251204,0.572392,0.491422,0.885109,0.198619,0.085143,0.178307
pre6m_cust_non_disc_amt,0.523949,0.290233,1.000000,0.456611,0.316239,0.197752,0.211577,0.282745,0.139843,0.240423,0.156977,0.547063,0.216703,0.193958,0.076953,0.240305
pre6m_cust_non_disc_cnt,0.298131,0.584062,0.456611,1.000000,0.209937,0.374180,0.118038,0.180767,0.159512,0.166463,0.382553,0.260328,0.411461,0.219909,0.076552,0.120173
pre6m_cust_disc_amt,0.973711,0.470719,0.316239,0.209937,1.000000,0.476252,0.673750,0.842883,0.432395,0.605658,0.301381,0.753789,0.391116,0.238724,0.119807,0.334589
pre6m_cust_disc_cnt,0.475038,0.971288,0.197752,0.374180,0.476252,1.000000,0.373618,0.518836,0.893065,0.238204,0.541819,0.485135,0.890616,0.162462,0.074837,0.168489
pre6m_cust_outbound_amt,0.655679,0.357307,0.211577,0.118038,0.673750,0.373618,1.000000,0.587448,0.329063,0.559802,0.294847,0.479780,0.282046,0.178058,0.080079,0.331681
pre6m_cust_online_amt,0.824611,0.500508,0.282745,0.180767,0.842883,0.518836,0.587448,1.000000,0.541052,0.440550,0.225599,0.615725,0.476899,0.220347,0.130589,0.251533
pre6m_cust_online_cnt,0.421737,0.822616,0.139843,0.159512,0.432395,0.893065,0.329063,0.541052,1.000000,0.147331,0.199754,0.497747,0.939017,0.111648,0.061795,0.110914
pre6m_cust_travel_amt,0.601484,0.251204,0.240423,0.166463,0.605658,0.238204,0.559802,0.440550,0.147331,1.000000,0.360391,0.262993,0.124364,0.201628,0.084155,0.329124


In [210]:
df1.drop(columns='pre6m_cust_roc_cnt',axis=1,inplace=True,errors='ignore')

# for f in drop_cols1:
#     grp1.remove(f)

df1.drop(columns=drop_cols1,axis=1,inplace=True,errors='ignore')

grp1.remove('pre6m_cust_disc_amt')

df1.drop(columns='pre6m_cust_disc_amt',axis=1,inplace=True,errors='ignore')

# pre6m_cust_roc_cnt, pre6m_cust_non_disc_amt, drop_cols1, pre6m_cust_disc_amt

In [211]:
for i in range(len(grp1)):

    y1 = df1.loc[:,grp1[i]]
    x1 = df1.loc[:,[f for f in grp1 if f!=grp1[i]]]
    
    model = sm.OLS(y1,x1)
    results = model.fit()
    rsq = results.rsquared
    vif = round(1/(1-rsq),2)
    
    print("R square value of {} column is {} keeping all other columns as features".format(grp1[i],(round(rsq,2))))
    print("Variance Inflation Factor of {} column is {} \n".format(grp1[i],vif))

R square value of pre6m_cust_spend column is 0.83 keeping all other columns as features
Variance Inflation Factor of pre6m_cust_spend column is 5.93 

R square value of pre6m_cust_non_disc_cnt column is 0.66 keeping all other columns as features
Variance Inflation Factor of pre6m_cust_non_disc_cnt column is 2.94 

R square value of pre6m_cust_disc_cnt column is 0.9 keeping all other columns as features
Variance Inflation Factor of pre6m_cust_disc_cnt column is 9.98 

R square value of pre6m_cust_outbound_amt column is 0.54 keeping all other columns as features
Variance Inflation Factor of pre6m_cust_outbound_amt column is 2.15 

R square value of pre6m_cust_online_amt column is 0.78 keeping all other columns as features
Variance Inflation Factor of pre6m_cust_online_amt column is 4.64 

R square value of pre6m_cust_online_cnt column is 0.86 keeping all other columns as features
Variance Inflation Factor of pre6m_cust_online_cnt column is 7.15 

R square value of pre6m_cust_myca_active 

In [176]:
train.isnull().sum()

cust_id                               4
profitable_flag                       4
count_accts                           4
cm_age                                4
flag_top_ed_spender                  52
flag_cust_fee_paid_6m                 4
pre6m_cust_spend                    619
pre6m_cust_roc_cnt                    4
pre6m_cust_non_disc_amt            1196
pre6m_cust_non_disc_cnt               4
pre6m_cust_disc_amt                 729
pre6m_cust_disc_cnt                   4
pre6m_cust_outbound_amt            2250
pre6m_cust_online_amt              1253
pre6m_cust_online_cnt                 4
pre6m_cust_travel_amt              1978
pre6m_cust_travel_cnt                 4
pre6m_cust_retail_amt               858
pre6m_cust_retail_cnt                 4
pre6m_cust_myca_active                4
pre6m_cust_mob_logins                 4
pre6m_total_mc_trs                    4
acq_sub_chn                         906
acq_type_grp                        876
fee_type_grp                          4


### One Hot Encoding

In [84]:
for f in categorical_cols:
    dummies = pd.get_dummies(x[str(f)])
    x = pd.concat([x,dummies],axis='columns')

### Label Encoding

In [37]:
categorical_cols

['acq_sub_chn', 'acq_type_grp', 'fee_type_grp']

In [39]:
# for col in categorical_cols:
    

In [40]:
x.head()

,count_accts,cm_age,flag_top_ed_spender,flag_cust_fee_paid_6m,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,pre6m_cust_disc_amt,pre6m_cust_disc_cnt,pre6m_cust_outbound_amt,pre6m_cust_online_amt,pre6m_cust_online_cnt,pre6m_cust_travel_amt,pre6m_cust_travel_cnt,pre6m_cust_retail_amt,pre6m_cust_retail_cnt,pre6m_cust_myca_active,pre6m_cust_mob_logins,pre6m_total_mc_trs,acq_sub_chn,acq_type_grp,fee_type_grp,Cust_tenure,pre6m_spend_active_ind,highly_utilized_ind,min_pay_ind,paid_in_full_ind,sum_total_line_amt,direct_debit_ind,referrals,spillover,self_accts,Customer Low Quality indicator,cdss_most_rcnt_prob,cust_max_credit_12m_amt,cust_max_remit_12m_amt,cbr_3_score,cnsumr_chrg_actv_cust_cnt,cnsumr_chrg_avg_credit_12m_amt,cnsumr_lend_actv_cust_cnt,cnsumr_lend_tot_util_ratio,sow_revol_avg_paydown_pct,sow_tot_annual_ext_pmt_amt,sow_tot_revol_bal_amt,sow_tot_revol_cnt,sow_tot_trans_bal_amt,sow_tot_trans_cnt,Affiliates,DM,Direct,E-Mails,E-mails,Email,In House,Interactive,MGM,Media,NaN0,Natural Referral,Natural Search,Non-PDB (Personalized),Outbound,Outsourced,PDB (Personalized),Paid Search,Partner,Partnerships - In Store,Partnerships - Other,Personalization,Proprietary - Direct Sales,Search,Unaddressed,Unreferred,NaN1,external,others,spill,full_fee,no_fee,reduced_fee,unknown
0,1.0,74.0,1.0,0.0,1518.0,122.0,811.0,96.0,707.0,26.0,-1.0,606.0,36.0,28.0,2.0,814.0,98.0,9.0,203.0,2.0,Unreferred,external,no_fee,63.0,1.0,0.0,0.0,1.0,80000.0,1.0,1.0,0.0,0.0,0.0,0.000,482.0,418.0,1376.0,0.0,-1.0,1.0,0.034044,0.0000,991.0,-1.0,0.0,-1.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
1,1.0,64.0,0.0,0.0,4807.0,354.0,1052.0,82.0,3756.0,272.0,673.0,3164.0,456.0,2291.0,44.0,1265.0,116.0,0.0,0.0,15.0,In House,others,no_fee,203.0,1.0,0.0,0.0,1.0,140000.0,1.0,1.0,1.0,0.0,0.0,0.000,2165.0,2165.0,1300.0,0.0,7.0,1.0,0.044086,0.0877,2.0,24.0,2.0,-1.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,1.0,75.0,1.0,0.0,2361.0,204.0,1351.0,118.0,1010.0,86.0,237.0,693.0,152.0,-1.0,0.0,1623.0,166.0,9.0,18.0,20.0,NaN0,NaN1,full_fee,201.0,1.0,0.0,0.0,1.0,32800.0,0.0,0.0,0.0,0.0,0.0,0.000,703.0,687.0,1376.0,0.0,-1.0,1.0,0.039613,0.0000,4828.0,-1.0,0.0,387.0,3.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,2.0,73.0,0.0,1.0,5452.0,142.0,1195.0,50.0,4257.0,92.0,803.0,1618.0,64.0,2760.0,54.0,1605.0,54.0,9.0,23.0,30.0,NaN0,NaN1,full_fee,286.0,1.0,0.0,0.0,1.0,100000.0,1.0,0.0,0.0,0.0,0.0,0.000,1225.0,1225.0,1372.0,1.0,493.0,1.0,0.001200,0.0000,4598.0,-1.0,0.0,295.0,4.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,2.0,68.0,0.0,0.0,170.0,34.0,88.0,20.0,82.0,14.0,10.0,10.0,4.0,-1.0,0.0,66.0,18.0,0.0,0.0,9.0,Affiliates,spill,no_fee,123.0,1.0,0.0,0.0,1.0,20800.0,1.0,0.0,0.0,0.0,0.0,0.001,51.0,35.0,1338.0,0.0,-1.0,2.0,0.003378,0.0168,2176.0,2606.0,5.0,143.0,13.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


### Bayesian Encoding

In [138]:
x.head()

,count_accts,cm_age,flag_top_ed_spender,flag_cust_fee_paid_6m,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,pre6m_cust_disc_amt,pre6m_cust_disc_cnt,pre6m_cust_outbound_amt,pre6m_cust_online_amt,pre6m_cust_online_cnt,pre6m_cust_travel_amt,pre6m_cust_travel_cnt,pre6m_cust_retail_amt,pre6m_cust_retail_cnt,pre6m_cust_myca_active,pre6m_cust_mob_logins,pre6m_total_mc_trs,acq_sub_chn,acq_type_grp,fee_type_grp,Cust_tenure,pre6m_spend_active_ind,highly_utilized_ind,min_pay_ind,paid_in_full_ind,sum_total_line_amt,direct_debit_ind,referrals,spillover,self_accts,Customer Low Quality indicator,cdss_most_rcnt_prob,cust_max_credit_12m_amt,cust_max_remit_12m_amt,cbr_3_score,cnsumr_chrg_actv_cust_cnt,cnsumr_chrg_avg_credit_12m_amt,cnsumr_lend_actv_cust_cnt,cnsumr_lend_tot_util_ratio,sow_revol_avg_paydown_pct,sow_tot_annual_ext_pmt_amt,sow_tot_revol_bal_amt,sow_tot_revol_cnt,sow_tot_trans_bal_amt,sow_tot_trans_cnt,Affiliates,DM,Direct,E-Mails,E-mails,Email,In House,Interactive,MGM,Media,NaN0,Natural Referral,Natural Search,Non-PDB (Personalized),Outbound,Outsourced,PDB (Personalized),Paid Search,Partner,Partnerships - In Store,Partnerships - Other,Personalization,Proprietary - Direct Sales,Search,Unaddressed,Unreferred,NaN1,external,others,spill,full_fee,no_fee,reduced_fee,unknown
0,1.0,74.0,1.0,0.0,1518.0,122.0,811.0,96.0,707.0,26.0,-1.0,606.0,36.0,28.0,2.0,814.0,98.0,9.0,203.0,2.0,Unreferred,external,no_fee,63.0,1.0,0.0,0.0,1.0,80000.0,1.0,1.0,0.0,0.0,0.0,0.000,482.0,418.0,1376.0,0.0,-1.0,1.0,0.034044,0.0000,991.0,-1.0,0.0,-1.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
1,1.0,64.0,0.0,0.0,4807.0,354.0,1052.0,82.0,3756.0,272.0,673.0,3164.0,456.0,2291.0,44.0,1265.0,116.0,0.0,0.0,15.0,In House,others,no_fee,203.0,1.0,0.0,0.0,1.0,140000.0,1.0,1.0,1.0,0.0,0.0,0.000,2165.0,2165.0,1300.0,0.0,7.0,1.0,0.044086,0.0877,2.0,24.0,2.0,-1.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,1.0,75.0,1.0,0.0,2361.0,204.0,1351.0,118.0,1010.0,86.0,237.0,693.0,152.0,-1.0,0.0,1623.0,166.0,9.0,18.0,20.0,NaN0,NaN1,full_fee,201.0,1.0,0.0,0.0,1.0,32800.0,0.0,0.0,0.0,0.0,0.0,0.000,703.0,687.0,1376.0,0.0,-1.0,1.0,0.039613,0.0000,4828.0,-1.0,0.0,387.0,3.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,2.0,73.0,0.0,1.0,5452.0,142.0,1195.0,50.0,4257.0,92.0,803.0,1618.0,64.0,2760.0,54.0,1605.0,54.0,9.0,23.0,30.0,NaN0,NaN1,full_fee,286.0,1.0,0.0,0.0,1.0,100000.0,1.0,0.0,0.0,0.0,0.0,0.000,1225.0,1225.0,1372.0,1.0,493.0,1.0,0.001200,0.0000,4598.0,-1.0,0.0,295.0,4.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,2.0,68.0,0.0,0.0,170.0,34.0,88.0,20.0,82.0,14.0,10.0,10.0,4.0,-1.0,0.0,66.0,18.0,0.0,0.0,9.0,Affiliates,spill,no_fee,123.0,1.0,0.0,0.0,1.0,20800.0,1.0,0.0,0.0,0.0,0.0,0.001,51.0,35.0,1338.0,0.0,-1.0,2.0,0.003378,0.0168,2176.0,2606.0,5.0,143.0,13.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [139]:
x.drop(columns=categorical_cols,axis=1,inplace=True)

### Dropping Columns Experiment

In [85]:
drop_cols1 = ['pre6m_cust_retail_cnt','pre6m_cust_retail_amt','pre6m_cust_travel_cnt','pre6m_cust_travel_amt']
corr_drop = ['pre6m_cust_roc_cnt','pre6m_cust_non_disc_amt','pre6m_cust_disc_amt']

x.drop(columns=drop_cols1,axis=1,errors='ignore',inplace=True)
x.drop(columns=corr_drop,axis=1,errors='ignore',inplace=True)

### TRAINING DATAFRAMES


In [ ]:
# x, x_tree_fea

### Feature Selection Methods

In [42]:
model = ExtraTreesClassifier()
model.fit(x, y)

ValueError: could not convert string to float: 'Unreferred'

In [43]:
# plt.figure(figsize=(20, 10))
# plt.title('XGB Most Important Features')
# plt.tight_layout()

feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_, x.columns)), columns=["Value", "Feature"])
feature_imp = feature_imp.sort_values(by="Value", ascending=False).iloc[:40]   # first 30 imp features
# ax = sns.barplot(x=feature_imp["Value"], y=feature_imp["Feature"].apply(lambda x: str(x)[:7]), color="tab:purple")

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [256]:
tree_fea_df = feature_imp[:32]
tree_fea = list(tree_fea_df['Feature'])

In [261]:
x_tree_fea = x[tree_fea]

In [312]:
len(tree_fea)

32

### Scaling Min Max

In [86]:
from sklearn.preprocessing import MinMaxScaler

In [326]:
sc1 = MinMaxScaler()
x_tree_fea = sc1.fit_transform(x_tree_fea)

In [88]:
x['Unreferred'].astype(int)
sc2 = MinMaxScaler()
x = sc2.fit(x)

ValueError: could not convert string to float: 'Unreferred'

In [43]:
from sklearn.model_selection import train_test_split

### TRAIN TEST SPLIT

In [44]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=17,stratify=y)

### Resampling Test Data

In [48]:
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

# tl = TomekLinks()
# X_tl, y_tl= tl.fit_sample(x_train, y_train)

# smote = SMOTE()
# X_sm, y_sm = smote.fit_sample(x_train, y_train)

nm = NearMiss()
X_res,y_res = nm.fit_sample(x,y)

ValueError: could not convert string to float: 'Unreferred'

### Splitting After Sampling


In [45]:
x_train,x_test,y_train,y_test = train_test_split(X_res,y_res,test_size=0.2,random_state=17,stratify=y_res)

NameError: name 'X_res' is not defined

In [332]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

### XGB

In [333]:
XGB = xgb.XGBClassifier(
    nthread=4,
    max_depth=12, 
    missing=-1,
    subsample=0.8,
    n_estimators=2000,
    learning_rate=0.02, 
    colsample_bytree=0.4, 
    eval_metric="aucpr",
    tree_method="hist" 
)

In [334]:
h = XGB.fit(x_train, y_train,    
    eval_set=[(x_test, y_test)],
    verbose=50,
    early_stopping_rounds=100,
)

[0]	validation_0-aucpr:0.22368
Will train until validation_0-aucpr hasn't improved in 100 rounds.
[50]	validation_0-aucpr:0.29684
[100]	validation_0-aucpr:0.29316
[150]	validation_0-aucpr:0.29100
Stopping. Best iteration:
[50]	validation_0-aucpr:0.29684



In [335]:
y_pred = XGB.predict(x_test)

In [336]:
from sklearn.metrics import f1_score

f1_score(y_pred,y_test)

In [337]:
list(y_pred).count(1)

17

In [338]:
list(y_test).count(1)

729

### XGB WITH SMOTE

In [339]:
h = XGB.fit(X_sm, y_sm,    
    eval_set=[(x_test, y_test)],
    verbose=50,
    early_stopping_rounds=100,
)

[0]	validation_0-aucpr:0.22470
Will train until validation_0-aucpr hasn't improved in 100 rounds.
[50]	validation_0-aucpr:0.25854
[100]	validation_0-aucpr:0.26852
[150]	validation_0-aucpr:0.26864
[200]	validation_0-aucpr:0.26918
[250]	validation_0-aucpr:0.27089
[300]	validation_0-aucpr:0.27233
[350]	validation_0-aucpr:0.27369
[400]	validation_0-aucpr:0.27193
[450]	validation_0-aucpr:0.27186
Stopping. Best iteration:
[352]	validation_0-aucpr:0.27396



In [340]:
y_pred = XGB.predict(x_test)

In [341]:
f1_score(y_pred,y_test)

0.11045828437132785

In [342]:
list(y_pred).count(1)

122

In [343]:
list(y_pred).count(0)

3797

In [344]:
list(y_test).count(0)

3190

### TABNET

In [316]:
# To check and print column with frequency greater than 1

import collections
print([item for item, count in collections.Counter(x.columns).items() if count > 1])

[]
